In [1]:
# Librerías estándar
import os
import gzip
import pickle
import zipfile
import json

# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Scikit-learn: Modelos, preprocesamiento y métricas
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score,
    recall_score, f1_score, confusion_matrix
)

### **Paso 1: Realice la limpieza de los datasets:**
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores
   de educación, agrupe estos valores en la categoría "others".
 - Renombre la columna "default payment next month" a "default"
 - Remueva la columna "ID".

In [2]:
# 1.1 Cargar los datos
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")

# 1.2 Organizar los datasets:

def limpiar(df):
    # Renombrar columna
    df = df.rename(columns={"default payment next month": "default"})
    # Eliminar columna 'ID'
    df.drop(columns=["ID"], inplace=True)
    # Modificar 'EDUCATION' (valores mayores a 4 se convierten en 4)
    df["EDUCATION"] = df["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
    # Asignar NaN a valores de 'EDUCATION' menores o iguales a 0
    df["EDUCATION"] = df["EDUCATION"].apply(lambda x: x if x > 0 else np.nan)
    # Asignar NaN a valores de 'MARRIAGE' menores o iguales a 0
    df["MARRIAGE"] = df["MARRIAGE"].apply(lambda x: x if x > 0 else np.nan)
    # Eliminar filas con valores faltantes
    df.dropna(inplace=True)
    return df

# Aplicar la función a los datasets
train_data = limpiar(train_data)
test_data = limpiar(test_data)


### **Paso 2: Divida los datasets en x_train, y_train, x_test, y_test.**

In [3]:
x_train = train_data.drop(columns=["default"])
y_train = train_data["default"]

x_test = test_data.drop(columns=["default"])
y_test = test_data["default"]

 ### **Paso 3: Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:**
 - Transforma las variables categoricas usando el método   one-hot-encoding.
- Descompone la matriz de entrada usando componentes principales. El pca usa todas las componentes.
- Escala la matriz de entrada al intervalo [0, 1].
- Selecciona las K columnas mas relevantes de la matrix de entrada.
- Ajusta una red neuronal tipo MLP.


In [21]:
colc = ['SEX','EDUCATION','MARRIAGE']
numc= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]

# Preprocesamiento: Transformación y Escalamiento

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), colc),
        ('num', StandardScaler(), numc)
    ]
)

k_best_selector = SelectKBest(score_func=f_classif)

# Crear el pipeline
pipeline=Pipeline(
    [
        ("preprocessor",preprocessor),
        ('feature_selection',SelectKBest(score_func=f_classif)),
        ('pca',PCA()),
        ('classifier',MLPClassifier(max_iter=15000,random_state=17))
        #('classifier',MLPClassifier(max_iter=10000,random_state=42))
    ]
)


# Precisión: 
pipeline.fit(x_train, y_train)
print("Precisión:", pipeline.score(x_test, y_test))

Precisión: 0.8281545829156922


### **Paso 4: Optimice los hiperparametros del pipeline usando validación cruzada. Use 10 splits para la validación cruzada. Use la función de precision balanceada para medir la precisión del modelo.**

In [5]:
#param_grid = {
#    'pca__n_components': [None],
#    'feature_selection__k': [20],
#    'mlp__hidden_layer_sizes': [(51, 31, 41)],
#    'mlp__alpha': [0.24],
#    'mlp__learning_rate_init': [0.0009]
#}
    

#grid_search = GridSearchCV(
#    pipeline, 
#    param_grid, 
#    cv=10, 
#    scoring='balanced_accuracy', 
#    n_jobs=-1)
#grid_search.fit(x_train, y_train)

In [22]:
param_grid = {
    'pca__n_components': [None],
    'feature_selection__k':[20],
    "classifier__hidden_layer_sizes": [(50, 30, 40,60)],
    'classifier__alpha': [0.26],
    #'classifier__solver':['lbfgs', 'adam'],
    #"classifier__activation": ['relu'],
    #"classifier__learning_rate": ['invscaling'],
    "classifier__learning_rate_init": [0.001],
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  
    cv=10,                                        
    n_jobs=-1,                                  
    verbose=1,
    refit=True                              
)
grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PAY_A...
                                                                          'PAY_AMT6'])])),
                                       ('feature_selection', SelectKBest()),
                                       ('pca', PCA()),
                                       ('classifier',
                                        MLPClassifier(max_iter=15000,
                                                      random_state=17))]),
             n_jobs=-1,
             param_grid={'classifier__alpha': [0.26],
                         'classifier__hidden_layer_sizes': [(50, 30, 40, 60)],
                         'classifier__learning_rate_init': [0.001],
                         'feature_selection__k': [20],
                         'pca__n_components': [None]},
             scoring='balanced_accuracy', verbose=1)

In [23]:
y_train_pred = grid_search.predict(x_train)
y_test_pred = grid_search.predict(x_test)

# Nota:
# Los score son diferentes si utilizo el best_estimator del modelo, pero
# las otras métricas no, porque por defecto utilizan el mejor modelo

print("Score train", grid_search.score(x_train, y_train), ">", 0.661,) 
print("Score test", grid_search.score(x_test, y_test), ">", 0.666,)
print()

train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_balanced_accuracy = balanced_accuracy_score(y_train, y_train_pred)

test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_test_pred)

train_cm = confusion_matrix(y_train, y_train_pred)
test_cm = confusion_matrix(y_test, y_test_pred)

print("train_precision", train_precision, ">", 0.691, train_precision > 0.691) 
print("train_balanced_accuracy", train_balanced_accuracy, ">", 0.661, train_balanced_accuracy > 0.661)
print("train_recall", train_recall, ">", 0.370, train_recall > 0.370)
print("train_f1", train_f1, ">", 0.482, train_f1 > 0.482) 
print()

print("test_precision", test_precision, ">", 0.673, test_precision > 0.673)
print("test_balanced_accuracy", test_balanced_accuracy, ">", 0.661, test_balanced_accuracy > 0.661)
print("test_recall", test_recall, ">", 0.370, test_recall > 0.370)
print("test_f1", test_f1, ">", 0.482, test_f1 > 0.482) 
print()

print(train_cm[0][0], 15440, train_cm[0][0] > 15440)
print(train_cm[1][1], 1735, train_cm[1][1] > 1735)
print(test_cm[0][0], 6710, test_cm[0][0] > 6710)
print(test_cm[1][1], 730, test_cm[1][1] > 730)

print('-------------------------')


Score train 0.6630025835547158 > 0.661
Score test 0.6695640976303335 > 0.666

train_precision 0.7025189924030388 > 0.691 True
train_balanced_accuracy 0.6630025835547158 > 0.661 True
train_recall 0.3718518518518519 > 0.37 True
train_f1 0.48629947412122887 > 0.482 True

test_precision 0.6846011131725418 > 0.673 True
test_balanced_accuracy 0.6695640976303335 > 0.661 True
test_recall 0.38719832109129065 > 0.37 True
test_f1 0.49463806970509383 > 0.482 True

15484 15440 True
1757 1735 True
6733 6710 True
738 730 True
-------------------------


### **Paso 5: Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz". Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.**

In [24]:
os.makedirs('../files/models', exist_ok=True)

with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

 ### **Paso 6: Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba.**

In [25]:
def cargar_modelo_y_predecir(data, modelo_path="../files/models/model.pkl.gz"):
    """
    Carga un modelo guardado en un archivo comprimido y realiza predicciones sobre los datos proporcionados.
    
    Args:
        data (array-like): Datos sobre los que se realizarán las predicciones.
        modelo_path (str): Ruta al archivo comprimido que contiene el modelo guardado.

    Returns:
        array-like: Predicciones realizadas por el modelo.
    """
    try:
        with gzip.open(modelo_path, "rb") as file:
            estimator = pickle.load(file)
        return estimator.predict(data)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo de modelo en la ruta especificada: {modelo_path}")
    except Exception as e:
        raise RuntimeError(f"Error al cargar el modelo o realizar predicciones: {e}")

# Uso de la función
y_train_pred = cargar_modelo_y_predecir(x_train)
y_test_pred = cargar_modelo_y_predecir(x_test)

In [27]:
def escribir_metricas(dict_metricas):
    models_dir = '../files/output'
    os.makedirs(models_dir, exist_ok=True)
    
    if os.path.exists('../files/output/metrics.json'):
        with open('../files/output/metrics.json', mode='r') as file:
            if len(file.readlines()) >= 4:
                os.remove('../files/output/metrics.json')
    
    with open('../files/output/metrics.json', mode='a') as file:
        file.write(str(dict_metricas).replace("'",'"')+"\n")

        
def eval_metrics(dataset,y_true, y_pred):
    accuracy = float(accuracy_score(y_true, y_pred))
    precision= float(precision_score(y_true, y_pred))
    balanced_accuracy = float(balanced_accuracy_score(y_true, y_pred))
    recall = float(recall_score(y_true, y_pred))
    f1 = float(f1_score(y_true, y_pred))
    metrics= {
        "type": "metrics",
        "dataset": dataset,
        "precision": precision,
        "balanced_accuracy": balanced_accuracy,
        "recall": recall,
        "f1_score": f1
    }
    
    escribir_metricas(metrics)

metrics_train = eval_metrics('train',y_train,y_train_pred)
metrics_test = eval_metrics('test',y_test,y_test_pred)

### **Paso 7: Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba.**

In [28]:
def matriz_confusion(dataset,y_true, y_pred):
    matriz = confusion_matrix(y_true, y_pred)
    matrix_confusion = {
        "type":"cm_matrix",
        "dataset": dataset,
        "true_0":{
            "predicted_0": int(matriz[0,0]),
            "predicted_1": None
        },
        "true_1":{
            "predicted_0": None,
            "predicted_1": int(matriz[1,1])
        }  
    }
    
    escribir_metricas(json.dumps(matrix_confusion))


metrics_train = matriz_confusion('train',y_train,y_train_pred)
metrics_test = matriz_confusion('test',y_test,y_test_pred)